In [42]:
import numpy as np
import pandas as pd
similarity_matrix = np.load('/data/chenxi/research/conference/TMIS/COTR/step3-bert_recall/CuBERT/outputs/en/2e-05_32_0.8/similarity_matrix.npy').T
job_df = pd.read_csv('/data/chenxi/research/conference/TMIS/COTR/step3-bert_recall/data/en/en.csv', sep='\t')
task_df = pd.read_csv('/data/chenxi/research/conference/TMIS/COTR/step3-bert_recall/data/en/task.csv', sep='\t')

task_map = pd.read_excel('/data/chenxi/research/conference/TMIS/COTR/step4-llm_select/results/en/en_results.xlsx')
task_map = task_map.groupby('task').agg(list)[['job_description']].reset_index()
task_df['task'] = task_df['DWA Title']
task_df = pd.merge(task_df, task_map, how='left').reset_index(drop=True)

index_list = np.argsort(-similarity_matrix)
sort_similarity_matrix = -np.sort(-similarity_matrix)
top_num = 20
task_df['sort_job'] = [i[:top_num] for i in index_list]
task_df['sort_job'] = task_df['sort_job'].apply(lambda x: [job_df['job_description'][i] for i in x])
task_df['sort_sim'] = [i[:top_num] for i in sort_similarity_matrix]
task_df['job_len'] = task_df['job_description'].apply(lambda x: len(x) if type(x) == type([]) else 0)
task_df

,Element Name,IWA Title,DWA Title,task,job_description,sort_job,sort_sim,job_len
0,Getting Information,Study details of artistic productions.,Review art or design materials.,Review art or design materials.,[Work closely with the entire creative team on...,[track inventoryMeet contractual conditions of...,"[0.99779266, 0.99777174, 0.9977128, 0.9975727,...",2
1,Getting Information,Study details of artistic productions.,Study details of musical compositions.,Study details of musical compositions.,[all underpinned in a tightly controlled finan...,[Provides administrative support to ASBC staff...,"[0.99807703, 0.9980697, 0.99802077, 0.997982, ...",2
2,Getting Information,Study details of artistic productions.,Review production information to determine cos...,Review production information to determine cos...,[Review detailed audit schedules for general l...,[Develop list of opportunity accounts with sal...,"[0.9981807, 0.99814713, 0.9980954, 0.998073, 0...",6
3,Getting Information,Study details of artistic productions.,Study scripts to determine project requirements.,Study scripts to determine project requirements.,[Collaborate with the senior investment team t...,[Develop a territory management plan to maximi...,"[0.99804944, 0.99795526, 0.9978592, 0.9978454,...",3
4,Getting Information,Study details of artistic productions.,Review audio or video recordings.,Review audio or video recordings.,[The position will manage data centers for Cog...,[Communicate with the Branch Supervisor to rec...,"[0.9982924, 0.99824494, 0.99823374, 0.9982139,...",5
...,...,...,...,...,...,...,...,...
2082,Monitoring and Controlling Resources,Monitor resources or inventories.,Monitor availability of equipment or supplies.,Monitor availability of equipment or supplies.,[Manage/Track inventory levels and usages for ...,"[Ability to use technical manuals, internet, v...","[0.99684584, 0.99669266, 0.9966585, 0.99652904...",4
2083,Monitoring and Controlling Resources,Monitor resources or inventories.,Inventory materials or equipment.,Inventory materials or equipment.,[Delivers incoming material not assigned to th...,[The above includes but is not limited to maki...,"[0.99692607, 0.99687165, 0.9967757, 0.9967735,...",1
2084,Monitoring and Controlling Resources,Monitor resources or inventories.,Inventory medical supplies or equipment.,Inventory medical supplies or equipment.,[collaborating with the nurse for minor and ur...,[Schedules preventative maintenance inspection...,"[0.9976418, 0.9975062, 0.9974739, 0.99739486, ...",7
2085,Monitoring and Controlling Resources,Monitor resources or inventories.,Monitor inventories of products or materials.,Monitor inventories of products or materials.,[Set appropriate supply policies to manage inv...,[Construction Supervisors must be able to read...,"[0.9973896, 0.99716353, 0.99714094, 0.9971299,...",3


In [55]:
save_df = task_df[~task_df['job_description'].isna()].drop('task', axis=1).sort_values(['Element Name', 'IWA Title', 'DWA Title', 'job_len'], ascending=[False, False, False, False])
save_df.to_excel('data/en/task_jd.xlsx', index=None)

In [46]:
num_clusters = 100

from sklearn.metrics.pairwise import cosine_distances

def compute_distance_matrix(matrix):
    # 计算余弦距离
    distance_matrix = cosine_distances(matrix)
    return distance_matrix

from scipy.cluster.hierarchy import linkage, fcluster

def hierarchical_clustering(distance_matrix, num_clusters=35):
    # 使用层次聚类算法
    Z = linkage(distance_matrix, method='ward')  # 'ward' 方法适用于最小化欧式距离平方
    # 将标签分成num_clusters个簇
    clusters = fcluster(Z, t=num_clusters, criterion='maxclust')
    return clusters

import numpy as np

def select_representative_labels(distance_matrix, clusters, num_labels=35):
    unique_clusters = np.unique(clusters)
    representative_labels = []
    
    for cluster_id in unique_clusters:
        cluster_indices = np.where(clusters == cluster_id)[0]
        avg_distances = np.mean(distance_matrix[cluster_indices][:, cluster_indices], axis=1)
        representative_label = cluster_indices[np.argmax(avg_distances)]
        representative_labels.append(representative_label)

    return representative_labels[:num_labels]

distance_matrix = compute_distance_matrix(similarity_matrix)

clusters = hierarchical_clustering(distance_matrix, num_clusters=num_clusters)

representative_labels = select_representative_labels(distance_matrix, clusters, num_labels=num_clusters)

print("Representative labels:", representative_labels)

/tmp/ipykernel_1658886/3951207207.py:14: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  Z = linkage(distance_matrix, method='ward')  # 'ward' 方法适用于最小化欧式距离平方


Representative labels: [323, 1268, 1184, 285, 1028, 1207, 964, 879, 1172, 1376, 234, 1373, 257, 824, 851, 1034, 1287, 966, 1033, 552, 939, 1980, 155, 1509, 1903, 1921, 2041, 1692, 1647, 1504, 223, 363, 1803, 1464, 1765, 1805, 1711, 1878, 1641, 601, 751, 1944, 1800, 1789, 600, 1671, 576, 1291, 897, 1092, 975, 252, 654, 938, 1277, 1884, 1165, 227, 141, 1282, 931, 1718, 1503, 1511, 2058, 102, 105, 2016, 491, 1395, 624, 217, 1956, 1678, 1526, 1654, 1912, 1897, 1659, 1651, 1864, 1523, 796, 828, 1911, 791, 1310, 909, 1520, 1874, 1444, 495, 390, 694, 11, 717, 1521, 172, 527, 790]


In [104]:
choose_task = task_df.loc[representative_labels]
choose_task = choose_task.sort_values(['job_len', 'Element Name', 'IWA Title'], ascending=[False, False, False]).reset_index(drop=True)
choose_task = choose_task.drop_duplicates('IWA Title')
choose_task = choose_task.explode('job_description')[['Element Name', 'IWA Title', 'DWA Title', 'job_description']]
choose_task.to_excel('data/en/choose_task_v1.xlsx', index=None)
choose_task

,Element Name,IWA Title,DWA Title,job_description
0,"Communicating with Supervisors, Peers, or Subo...",Communicate with others about operational plan...,Liaise between departments or other groups to ...,Collaborate cross-functionally with internal t...
0,"Communicating with Supervisors, Peers, or Subo...",Communicate with others about operational plan...,Liaise between departments or other groups to ...,Coordinate and interface on a regular basis wi...
0,"Communicating with Supervisors, Peers, or Subo...",Communicate with others about operational plan...,Liaise between departments or other groups to ...,• Coordinating and liaison with the other cros...
0,"Communicating with Supervisors, Peers, or Subo...",Communicate with others about operational plan...,Liaise between departments or other groups to ...,This role involves close coordination with var...
0,"Communicating with Supervisors, Peers, or Subo...",Communicate with others about operational plan...,Liaise between departments or other groups to ...,Manage ESA’s Information Sharing and Analysis ...
...,...,...,...,...
94,Handling and Moving Objects,Apply materials to fill gaps or imperfections.,Apply material to fill gaps in surfaces.,NaN
95,Handling and Moving Objects,Apply hygienic or cosmetic agents to skin or h...,Apply makeup to alter or enhance appearance.,NaN
96,Handling and Moving Objects,Adjust equipment to ensure adequate performance.,Adjust flow of electricity to tools or product...,NaN
97,Controlling Machines and Processes,Operate agricultural or forestry equipment.,Operate grounds maintenance equipment.,NaN


In [112]:
large_task_list = task_df[task_df['job_len']>20].index.tolist()
choose_task2 = task_df.loc[list(set(large_task_list) - set(representative_labels))]
choose_task2 = choose_task2.sort_values([ 'job_len', 'Element Name', 'IWA Title', 'DWA Title'], ascending=[False, False, False, False]).reset_index(drop=True)
choose_task2 = choose_task2.explode('sort_job')[['Element Name', 'IWA Title', 'DWA Title', 'sort_job']]
choose_task2.to_excel('data/en/choose_task_v2.xlsx', index=None)
choose_task2

,Element Name,IWA Title,DWA Title,sort_job
0,Thinking Creatively,Develop business or marketing plans.,Develop marketing plans or strategies.,Assist with special projects and initiatives a...
0,Thinking Creatively,Develop business or marketing plans.,Develop marketing plans or strategies.,Achieve service and sales goals and objectives...
0,Thinking Creatively,Develop business or marketing plans.,Develop marketing plans or strategies.,Generate Annual Site-Based Expenditure ReportA...
0,Thinking Creatively,Develop business or marketing plans.,Develop marketing plans or strategies.,"Help plan, lead, and coordinate all activities..."
0,Thinking Creatively,Develop business or marketing plans.,Develop marketing plans or strategies.,"Effectively engage commercial counterparts, id..."
...,...,...,...,...
35,Assisting and Caring for Others,Assist healthcare practitioners during medical...,Assist healthcare practitioners during examina...,"Assessing, planning, organizing and overseeing..."
35,Assisting and Caring for Others,Assist healthcare practitioners during medical...,Assist healthcare practitioners during examina...,Demonstrates UPMC’s commitment to the Core Val...
35,Assisting and Caring for Others,Assist healthcare practitioners during medical...,Assist healthcare practitioners during examina...,Provides professional coordination of mental h...
35,Assisting and Caring for Others,Assist healthcare practitioners during medical...,Assist healthcare practitioners during examina...,Serve as the main point of contact for in-kind...


In [115]:
choose_task_list = ["Contact current or potential customers to promote products or services.","Evaluate applicable laws and regulations to determine impact on organizational activities.","Confer with managers to make operational decisions.","Communicate with customers to resolve complaints or ensure satisfaction.","Develop promotional strategies or plans."]
choose_task3 = task_df[task_df['task'].isin(choose_task_list)]
choose_task3 = choose_task3.sort_values([ 'job_len', 'Element Name', 'IWA Title', 'DWA Title'], ascending=[False, False, False, False]).reset_index(drop=True)
choose_task3_1 = choose_task3.explode('job_description')[['Element Name', 'IWA Title', 'DWA Title', 'job_description']]
choose_task3_2 = choose_task3.explode('sort_job')[['Element Name', 'IWA Title', 'DWA Title', 'sort_job']]
choose_task3_2.columns = choose_task3_1.columns
choose_task3 = pd.concat([choose_task3_1, choose_task3_2])
choose_task3.sort_values(['Element Name', 'IWA Title', 'DWA Title']).drop_duplicates()
choose_task3.to_excel('data/en/choose_task_v3.xlsx', index=None)
choose_task3

,Element Name,IWA Title,DWA Title,job_description
0,Selling or Influencing Others,"Promote products, services, or programs.",Contact current or potential customers to prom...,"Spend majority of time in the field, on sales ..."
0,Selling or Influencing Others,"Promote products, services, or programs.",Contact current or potential customers to prom...,Assist in executing marketing client campaigns...
0,Selling or Influencing Others,"Promote products, services, or programs.",Contact current or potential customers to prom...,"Prospect new business primarily by phone, mail..."
0,Selling or Influencing Others,"Promote products, services, or programs.",Contact current or potential customers to prom...,• Develop new suppliers and potential customer...
0,Selling or Influencing Others,"Promote products, services, or programs.",Contact current or potential customers to prom...,Collaborates with sales and support groups to ...
...,...,...,...,...
4,"Communicating with Supervisors, Peers, or Subo...",Communicate with others about operational plan...,Confer with managers to make operational decis...,Effectively communicate voice of customer (VOC...
4,"Communicating with Supervisors, Peers, or Subo...",Communicate with others about operational plan...,Confer with managers to make operational decis...,Define and Develop strategy by identifying tec...
4,"Communicating with Supervisors, Peers, or Subo...",Communicate with others about operational plan...,Confer with managers to make operational decis...,Cultivate new and strengthen existing relation...
4,"Communicating with Supervisors, Peers, or Subo...",Communicate with others about operational plan...,Confer with managers to make operational decis...,Help review data operations workflows and iden...


In [114]:
large_task_list = task_df[(task_df['job_len']>10) & (task_df['job_len']<=20)].index.tolist()
choose_task2 = task_df.loc[list(set(large_task_list) - set(representative_labels))]
choose_task2 = choose_task2.sort_values([ 'job_len', 'Element Name', 'IWA Title', 'DWA Title'], ascending=[False, False, False, False]).reset_index(drop=True)
choose_task2 = choose_task2.explode('sort_job')[['Element Name', 'IWA Title', 'DWA Title', 'sort_job']]
choose_task2.to_excel('data/en/choose_task_v4.xlsx', index=None)
choose_task2

,Element Name,IWA Title,DWA Title,sort_job
0,Staffing Organizational Units,Perform recruiting or hiring activities.,Hire personnel.,Corporate communications - Promote ESG events ...
0,Staffing Organizational Units,Perform recruiting or hiring activities.,Hire personnel.,Work with the board of directors to articulate...
0,Staffing Organizational Units,Perform recruiting or hiring activities.,Hire personnel.,job requisitionsPlan and coordinate recruitmen...
0,Staffing Organizational Units,Perform recruiting or hiring activities.,Hire personnel.,Build a network of KOL Royal Canin advocates a...
0,Staffing Organizational Units,Perform recruiting or hiring activities.,Hire personnel.,"• Conduct product demos, training sessions, an..."
...,...,...,...,...
129,Analyzing Data or Information,Analyze business or financial risks.,Analyze risks to minimize losses or damages.,Work closely with and assist Account Manager t...
129,Analyzing Data or Information,Analyze business or financial risks.,Analyze risks to minimize losses or damages.,Provide additional administrative support for ...
129,Analyzing Data or Information,Analyze business or financial risks.,Analyze risks to minimize losses or damages.,Prepares ad-hoc financial reports by collectin...
129,Analyzing Data or Information,Analyze business or financial risks.,Analyze risks to minimize losses or damages.,Generate sales among customer accountsAnswer c...


In [ ]:
import pandas as pd
pd.read_excel('')